In [1]:
import pandas as pd
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players, teams
from pprint import pprint 
import time, os, json


In [2]:
# Select all players id
lst_players = players.get_players()
# Create a DataFrame with all players id
player = pd.DataFrame({"PLAYER_ID":[x['id'] for x in lst_players]})
player.head()

# Create a DataFrame to store players seasons 
seasons = pd.DataFrame(columns=["PLAYER_ID","SEASON_ID"])


In [3]:
# Create all player columns 
player['PERSON_ID'] = None
player['FIRST_NAME'] = None
player['LAST_NAME'] = None
player['FULL_NAME'] = None
player['DISPLAY_LAST_COMMA_FIRST'] = None
player['DISPLAY_FI_LAST'] = None
player['BIRTHDATE'] = None
player['SCHOOL'] = None
player['COUNTRY'] = None
player['LAST_AFFILIATION'] = None
player['HEIGHT'] = None
player['WEIGHT'] = None
player['SEASON_EXP'] = None
player['JERSEY'] = None
player['POSITION'] = None
player['ROSTERSTATUS'] = None
player['TEAM_ID'] = None
player['TEAM_NAME'] = None
player['TEAM_ABBREVIATION'] = None
player['TEAM_CODE'] = None
player['TEAM_CITY'] = None
player['PLAYERCODE'] = None
player['FROM_YEAR'] = None
player['TO_YEAR'] = None
player['DLEAGUE_FLAG'] = None
player['NBA_FLAG'] = None
player['GAMES_PLAYED_FLAG'] = None
player['DRAFT_YEAR'] = None
player['DRAFT_ROUND'] = None
player['DRAFT_NUMBER'] = None
player['IS_ACTIVE'] = False
player['affiliation_state'] = None
player['affiliation_country'] = None
# 
player['PLAYER_NAME'] = None
player['TimeFrame'] = None
player['PTS'] = None
player['AST'] = None
player['REB'] = None
player['ALL_STAR_APPEARANCES'] = None

custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}


In [4]:
count = 0 
for index, row in player.iterrows(): 
    count += 1
    time.sleep(2)
    print(f"Processing id = {row['PLAYER_ID']} - Record {count}")
    try: 

        response = commonplayerinfo.CommonPlayerInfo(player_id=row['PLAYER_ID'], headers=custom_headers, timeout=100).get_dict()

        player_info = response['resultSets'][0]['rowSet'][0]

        player.loc[index, 'PERSON_ID']                  = player_info[0]
        player.loc[index, 'FIRST_NAME']                 = player_info[1]
        player.loc[index, 'LAST_NAME']                  = player_info[2]
        player.loc[index, 'FULL_NAME']                  = player_info[3]
        player.loc[index, 'DISPLAY_LAST_COMMA_FIRST']   = player_info[4]
        player.loc[index, 'DISPLAY_FI_LAST']            = player_info[5]
        player.loc[index, 'BIRTHDATE']                  = player_info[6]
        player.loc[index, 'SCHOOL']                     = player_info[7]
        player.loc[index, 'COUNTRY']                    = player_info[8]
        player.loc[index, 'LAST_AFFILIATION']           = player_info[9]
        player.loc[index, 'HEIGHT']                     = player_info[10]
        player.loc[index, 'WEIGHT']                     = player_info[11]
        player.loc[index, 'SEASON_EXP']                 = player_info[12]
        player.loc[index, 'JERSEY']                     = player_info[13]
        player.loc[index, 'POSITION']                   = player_info[14]
        player.loc[index, 'ROSTERSTATUS']               = player_info[15]
        player.loc[index, 'TEAM_ID']                    = player_info[16]
        player.loc[index, 'TEAM_NAME']                  = player_info[17]
        player.loc[index, 'TEAM_ABBREVIATION']          = player_info[18]
        player.loc[index, 'TEAM_CODE']                  = player_info[19]
        player.loc[index, 'TEAM_CITY']                  = player_info[20]
        player.loc[index, 'PLAYERCODE']                 = player_info[21]
        player.loc[index, 'FROM_YEAR']                  = player_info[22]
        player.loc[index, 'TO_YEAR']                    = player_info[23]
        player.loc[index, 'DLEAGUE_FLAG']               = player_info[24]
        player.loc[index, 'NBA_FLAG']                   = player_info[25]
        player.loc[index, 'GAMES_PLAYED_FLAG']          = player_info[26]
        player.loc[index, 'DRAFT_YEAR']                 = player_info[27]
        player.loc[index, 'DRAFT_ROUND']                = player_info[28]
        player.loc[index, 'DRAFT_NUMBER']               = player_info[29]

        # Split the affiliation if is filled 
        affiliation  = player_info[9]
        if (affiliation is None):
              affiliation = ""
        if (len(affiliation.split("/")) == 2):
            player.loc[index, 'affiliation_state'] = affiliation.split("/")[0] 
            player.loc[index, 'affiliation_country'] = affiliation.split("/")[1]   

        # Complementary Data:
        # Check whether data exist.
        if len(response['resultSets'][1]['rowSet']) > 0:
            complement = response['resultSets'][1]['rowSet'][0]
            player.loc[index, 'PLAYER_NAME']          = complement[1]
            player.loc[index, 'TimeFrame']            = complement[2]
            player.loc[index, 'PTS']                  = complement[3]
            player.loc[index, 'AST']                  = complement[4]
            player.loc[index, 'REB']                  = complement[5]
            player.loc[index, 'ALL_STAR_APPEARANCES'] = complement[6]

        # LOOPING TO GET ALL SEASONS TO THE CURRENT PLAYER AND SAVE IN A SEPARATED DF
        seasons_resp = response['resultSets'][2]['rowSet']
        for season in seasons_resp:
          seasons = seasons.append({'PLAYER_ID':row['PLAYER_ID'], 'SEASON_ID':season[0]}, ignore_index=True)

    except:    
        print(f"We've got an error when processing id={row['PLAYER_ID']}.")

#     if (count == 10):
#         break
                  
    if (count % 100 == 0):
        time.sleep(2)

player.head(20)

Processing id = 76001 - Record 1
Processing id = 76002 - Record 2
Processing id = 76003 - Record 3
Processing id = 51 - Record 4
Processing id = 1505 - Record 5
Processing id = 949 - Record 6
Processing id = 76005 - Record 7
Processing id = 76006 - Record 8
Processing id = 76007 - Record 9
Processing id = 203518 - Record 10
Processing id = 101165 - Record 11
Processing id = 76008 - Record 12
Processing id = 76009 - Record 13
Processing id = 76010 - Record 14
Processing id = 203112 - Record 15
Processing id = 76011 - Record 16
Processing id = 76012 - Record 17
Processing id = 200801 - Record 18
Processing id = 1629121 - Record 19
Processing id = 203919 - Record 20
Processing id = 149 - Record 21
Processing id = 203500 - Record 22
Processing id = 912 - Record 23
Processing id = 1628389 - Record 24
Processing id = 1629061 - Record 25
Processing id = 76015 - Record 26
Processing id = 202399 - Record 27
Processing id = 201167 - Record 28
Processing id = 200772 - Record 29
Processing id = 76

Processing id = 76124 - Record 239
Processing id = 201587 - Record 240
Processing id = 76125 - Record 241
Processing id = 76126 - Record 242
Processing id = 2437 - Record 243
Processing id = 201573 - Record 244
Processing id = 76127 - Record 245
Processing id = 203382 - Record 246
Processing id = 76128 - Record 247
Processing id = 203145 - Record 248
Processing id = 76129 - Record 249
Processing id = 203078 - Record 250
Processing id = 76130 - Record 251
Processing id = 76131 - Record 252
Processing id = 2602 - Record 253
Processing id = 1627736 - Record 254
Processing id = 201563 - Record 255
Processing id = 76133 - Record 256
Processing id = 201958 - Record 257
Processing id = 76134 - Record 258
Processing id = 1133 - Record 259
Processing id = 76136 - Record 260
Processing id = 76137 - Record 261
Processing id = 76138 - Record 262
Processing id = 76139 - Record 263
Processing id = 76140 - Record 264
Processing id = 76141 - Record 265
Processing id = 201158 - Record 266
Processing id

Processing id = 76266 - Record 473
Processing id = 418 - Record 474
Processing id = 2810 - Record 475
Processing id = 1626148 - Record 476
Processing id = 76284 - Record 477
Processing id = 201628 - Record 478
Processing id = 1628971 - Record 479
Processing id = 359 - Record 480
Processing id = 2245 - Record 481
Processing id = 76270 - Record 482
Processing id = 244 - Record 483
Processing id = 200793 - Record 484
Processing id = 201974 - Record 485
Processing id = 2484 - Record 486
Processing id = 2081 - Record 487
Processing id = 76272 - Record 488
Processing id = 76273 - Record 489
Processing id = 1871 - Record 490
Processing id = 76275 - Record 491
Processing id = 203913 - Record 492
Processing id = 1627759 - Record 493
Processing id = 76276 - Record 494
Processing id = 2208 - Record 495
Processing id = 2198 - Record 496
Processing id = 76278 - Record 497
Processing id = 76279 - Record 498
Processing id = 203485 - Record 499
Processing id = 992 - Record 500
Processing id = 203900 -

Processing id = 1127 - Record 708
Processing id = 2049 - Record 709
Processing id = 76402 - Record 710
Processing id = 2043 - Record 711
Processing id = 76403 - Record 712
Processing id = 1628499 - Record 713
Processing id = 76404 - Record 714
Processing id = 76406 - Record 715
Processing id = 1569 - Record 716
Processing id = 76407 - Record 717
Processing id = 76408 - Record 718
Processing id = 76409 - Record 719
Processing id = 76410 - Record 720
Processing id = 76411 - Record 721
Processing id = 715 - Record 722
Processing id = 202708 - Record 723
Processing id = 76412 - Record 724
Processing id = 934 - Record 725
Processing id = 76413 - Record 726
Processing id = 76414 - Record 727
Processing id = 76415 - Record 728
Processing id = 902 - Record 729
Processing id = 2044 - Record 730
Processing id = 76416 - Record 731
Processing id = 76418 - Record 732
Processing id = 76421 - Record 733
Processing id = 1530 - Record 734
Processing id = 2260 - Record 735
Processing id = 2215 - Record 

Processing id = 2568 - Record 943
Processing id = 1629116 - Record 944
Processing id = 960 - Record 945
Processing id = 203521 - Record 946
Processing id = 76552 - Record 947
Processing id = 76553 - Record 948
Processing id = 1628610 - Record 949
Processing id = 725 - Record 950
Processing id = 76555 - Record 951
Processing id = 2736 - Record 952
Processing id = 76556 - Record 953
Processing id = 76557 - Record 954
Processing id = 202458 - Record 955
Processing id = 76558 - Record 956
Processing id = 76559 - Record 957
Processing id = 1629094 - Record 958
Processing id = 76560 - Record 959
Processing id = 76561 - Record 960
Processing id = 76568 - Record 961
Processing id = 1628978 - Record 962
Processing id = 1760 - Record 963
Processing id = 1627767 - Record 964
Processing id = 1628977 - Record 965
Processing id = 2564 - Record 966
Processing id = 200821 - Record 967
Processing id = 200799 - Record 968
Processing id = 2424 - Record 969
Processing id = 2079 - Record 970
Processing id 

...

Processing id = 78225 - Record 3685
Processing id = 78226 - Record 3686
Processing id = 201578 - Record 3687
Processing id = 1629016 - Record 3688
Processing id = 78227 - Record 3689
Processing id = 771 - Record 3690
Processing id = 280 - Record 3691
Processing id = 78230 - Record 3692
Processing id = 78231 - Record 3693
Processing id = 201168 - Record 3694
Processing id = 78232 - Record 3695
Processing id = 84 - Record 3696
Processing id = 78233 - Record 3697
Processing id = 78234 - Record 3698
Processing id = 78235 - Record 3699
Processing id = 1756 - Record 3700
Processing id = 711 - Record 3701
Processing id = 78237 - Record 3702
Processing id = 78238 - Record 3703
Processing id = 78239 - Record 3704
Processing id = 78240 - Record 3705
Processing id = 78241 - Record 3706
Processing id = 317 - Record 3707
Processing id = 78243 - Record 3708
Processing id = 203917 - Record 3709
Processing id = 78244 - Record 3710
Processing id = 78245 - Record 3711
Processing id = 1099 - Record 3712


Processing id = 78363 - Record 3913
Processing id = 78364 - Record 3914
Processing id = 1626157 - Record 3915
Processing id = 78365 - Record 3916
Processing id = 78366 - Record 3917
Processing id = 78367 - Record 3918
Processing id = 1714 - Record 3919
Processing id = 718 - Record 3920
Processing id = 1629018 - Record 3921
Processing id = 2244 - Record 3922
Processing id = 78651 - Record 3923
Processing id = 1629019 - Record 3924
Processing id = 78368 - Record 3925
Processing id = 78369 - Record 3926
Processing id = 2456 - Record 3927
Processing id = 2054 - Record 3928
Processing id = 78370 - Record 3929
Processing id = 78371 - Record 3930
Processing id = 2401 - Record 3931
Processing id = 78372 - Record 3932
Processing id = 201169 - Record 3933
Processing id = 141 - Record 3934
Processing id = 78374 - Record 3935
Processing id = 200782 - Record 3936
Processing id = 78375 - Record 3937
Processing id = 101142 - Record 3938
Processing id = 1726 - Record 3939
Processing id = 2045 - Record

Processing id = 2753 - Record 4141
Processing id = 28 - Record 4142
Processing id = 78497 - Record 4143
Processing id = 201238 - Record 4144
Processing id = 770 - Record 4145
Processing id = 78499 - Record 4146
Processing id = 201566 - Record 4147
Processing id = 78500 - Record 4148
Processing id = 78501 - Record 4149
Processing id = 101156 - Record 4150
Processing id = 941 - Record 4151
Processing id = 1545 - Record 4152
Processing id = 78504 - Record 4153
Processing id = 1755 - Record 4154
Processing id = 78506 - Record 4155
Processing id = 201591 - Record 4156
Processing id = 1628401 - Record 4157
Processing id = 78507 - Record 4158
Processing id = 78508 - Record 4159
Processing id = 78509 - Record 4160
Processing id = 1751 - Record 4161
Processing id = 200778 - Record 4162
Processing id = 78510 - Record 4163
Processing id = 1627855 - Record 4164
Processing id = 1839 - Record 4165
Processing id = 2206 - Record 4166
Processing id = 78512 - Record 4167
Processing id = 203091 - Record 

Processing id = 201152 - Record 4370
Processing id = 1937 - Record 4371
Processing id = 1629027 - Record 4372
Processing id = 78638 - Record 4373
Processing id = 78639 - Record 4374
Processing id = 203469 - Record 4375
Processing id = 78640 - Record 4376
Processing id = 78641 - Record 4377
Processing id = 78642 - Record 4378
Processing id = 202545 - Record 4379
Processing id = 203092 - Record 4380
Processing id = 78643 - Record 4381
Processing id = 78644 - Record 4382
Processing id = 1917 - Record 4383
Processing id = 1627753 - Record 4384
Processing id = 678 - Record 4385
Processing id = 2583 - Record 4386
Processing id = 1627757 - Record 4387
Processing id = 1627835 - Record 4388
Processing id = 1627790 - Record 4389
Processing id = 78647 - Record 4390
Processing id = 78648 - Record 4391
Processing id = 1627826 - Record 4392
Processing id = 78650 - Record 4393


,PLAYER_ID,PERSON_ID,FIRST_NAME,LAST_NAME,FULL_NAME,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,BIRTHDATE,SCHOOL,COUNTRY,...,DRAFT_NUMBER,IS_ACTIVE,affiliation_state,affiliation_country,PLAYER_NAME,TimeFrame,PTS,AST,REB,ALL_STAR_APPEARANCES
0,76001,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,1968-06-24T00:00:00,Duke,USA,...,25,False,Duke,USA,Alaa Abdelnaby,career,5.7,0.3,3.3,0
1,76002,76002,Zaid,Abdul-Aziz,Zaid Abdul-Aziz,"Abdul-Aziz, Zaid",Z. Abdul-Aziz,1946-04-07T00:00:00,Iowa State,USA,...,5,False,Iowa State,USA,Zaid Abdul-Aziz,career,9,1.2,8,0
2,76003,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,1947-04-16T00:00:00,California-Los Angeles,USA,...,1,False,UCLA,USA,Kareem Abdul-Jabbar,career,24.6,3.6,11.2,18
3,51,51,Mahmoud,Abdul-Rauf,Mahmoud Abdul-Rauf,"Abdul-Rauf, Mahmoud",M. Abdul-Rauf,1969-03-09T00:00:00,Louisiana State,USA,...,3,False,Louisiana State,USA,Mahmoud Abdul-Rauf,career,14.6,3.5,1.9,0
4,1505,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,1974-11-03T00:00:00,San Jose State,France,...,11,False,San Jose State,France,Tariq Abdul-Wahad,career,7.8,1.1,3.3,0
5,949,949,Shareef,Abdur-Rahim,Shareef Abdur-Rahim,"Abdur-Rahim, Shareef",S. Abdur-Rahim,1976-12-11T00:00:00,California,USA,...,3,False,California,USA,Shareef Abdur-Rahim,career,18.1,2.5,7.5,1
6,76005,76005,Tom,Abernethy,Tom Abernethy,"Abernethy, Tom",T. Abernethy,1954-05-06T00:00:00,Indiana,USA,...,43,False,Indiana,USA,Tom Abernethy,career,5.6,1.2,3.2,0
7,76006,76006,Forest,Able,Forest Able,"Able, Forest",F. Able,1932-07-27T00:00:00,Western Kentucky,USA,...,None,False,Western Kentucky; Louisville,USA,Forest Able,career,0,1,1,0
8,76007,76007,John,Abramovic,John Abramovic,"Abramovic, John",J. Abramovic,1919-02-09T00:00:00,Salem International,USA,...,Undrafted,False,Salem (NC),USA,John Abramovic,career,9.5,0.7,None,0
9,203518,203518,Alex,Abrines,Alex Abrines,"Abrines, Alex",A. Abrines,1993-08-01T00:00:00,None,Spain,...,32,False,Spain,Spain,Alex Abrines,career,5.3,0.5,1.4,0


In [5]:
seasons.head(50)

,PLAYER_ID,SEASON_ID
0,76001,21990
1,76001,41990
2,76001,21991
3,76001,41991
4,76001,21992
5,76001,41992
6,76001,21993
7,76001,21994
8,76002,21968
9,76002,21969


In [6]:
# Assign the players' file to a variable 
players_file = os.path.join(".", "Resources", "players2.csv")
player.to_csv(players_file, encoding="utf-8", index=False)

In [7]:
# Assign the seasons' file to a variable 
seasons_file = os.path.join(".", "Resources", "seasons2.csv")
seasons.to_csv(seasons_file, encoding="utf-8", index=False)

In [8]:
# # Request for Player Id with error connection
# player_err = pd.DataFrame({'PLAYER_ID':[1626122 ,1626187 ,1628610 ,1629008 ,1629117 ,1629129 ,1629134 ,1629167, \
#                                         1629341 ,201195 ,201987 ,201998 ,202067 ,202070 ,202221 ,202238 ,202358 ,202364, \
#                                         202375 ,202392 ,203464 , 90000 ,986]})
# response = None
# for index, row in player_err.iterrows(): 
#     time.sleep(2)
#     print(f"Processing id = {row['PLAYER_ID']}")
# #     try: 

#     response = commonplayerinfo.CommonPlayerInfo(player_id=row['PLAYER_ID'], headers=custom_headers, timeout=100).get_dict()

#     player_info = response['resultSets'][0]['rowSet'][0]

#     player.loc[index, 'PERSON_ID']                  = player_info[0]
#     player.loc[index, 'FIRST_NAME']                 = player_info[1]
#     player.loc[index, 'LAST_NAME']                  = player_info[2]
#     player.loc[index, 'FULL_NAME']                  = player_info[3]
#     player.loc[index, 'DISPLAY_LAST_COMMA_FIRST']   = player_info[4]
#     player.loc[index, 'DISPLAY_FI_LAST']            = player_info[5]
#     player.loc[index, 'BIRTHDATE']                  = player_info[6]
#     player.loc[index, 'SCHOOL']                     = player_info[7]
#     player.loc[index, 'COUNTRY']                    = player_info[8]
#     player.loc[index, 'LAST_AFFILIATION']           = player_info[9]
#     player.loc[index, 'HEIGHT']                     = player_info[10]
#     player.loc[index, 'WEIGHT']                     = player_info[11]
#     player.loc[index, 'SEASON_EXP']                 = player_info[12]
#     player.loc[index, 'JERSEY']                     = player_info[13]
#     player.loc[index, 'POSITION']                   = player_info[14]
#     player.loc[index, 'ROSTERSTATUS']               = player_info[15]
#     player.loc[index, 'TEAM_ID']                    = player_info[16]
#     player.loc[index, 'TEAM_NAME']                  = player_info[17]
#     player.loc[index, 'TEAM_ABBREVIATION']          = player_info[18]
#     player.loc[index, 'TEAM_CODE']                  = player_info[19]
#     player.loc[index, 'TEAM_CITY']                  = player_info[20]
#     player.loc[index, 'PLAYERCODE']                 = player_info[21]
#     player.loc[index, 'FROM_YEAR']                  = player_info[22]
#     player.loc[index, 'TO_YEAR']                    = player_info[23]
#     player.loc[index, 'DLEAGUE_FLAG']               = player_info[24]
#     player.loc[index, 'NBA_FLAG']                   = player_info[25]
#     player.loc[index, 'GAMES_PLAYED_FLAG']          = player_info[26]
#     player.loc[index, 'DRAFT_YEAR']                 = player_info[27]
#     player.loc[index, 'DRAFT_ROUND']                = player_info[28]
#     player.loc[index, 'DRAFT_NUMBER']               = player_info[29]

#     # Split the affiliation if is filled 
#     affiliation  = player_info[9]
#     if (affiliation is None):
#           affiliation = ""
#     if (len(affiliation.split("/")) == 2):
#         player.loc[index, 'affiliation_state'] = affiliation.split("/")[0] 
#         player.loc[index, 'affiliation_country'] = affiliation.split("/")[1]   

#     # Complementary Data:
#     # Check whether data exist.
#     if len(response['resultSets'][1]['rowSet']) > 0:
#         complement = response['resultSets'][1]['rowSet'][0]
#         player.loc[index, 'PLAYER_NAME']          = complement[1]
#         player.loc[index, 'TimeFrame']            = complement[2]
#         player.loc[index, 'PTS']                  = complement[3]
#         player.loc[index, 'AST']                  = complement[4]
#         player.loc[index, 'REB']                  = complement[5]
#         player.loc[index, 'ALL_STAR_APPEARANCES'] = complement[6]

#     # LOOPING TO GET ALL SEASONS TO THE CURRENT PLAYER AND SAVE IN A SEPARATED DF
#     seasons_resp = response['resultSets'][2]['rowSet']
#     for season in seasons_resp:
#       seasons = seasons.append({'PLAYER_ID':row['PLAYER_ID'], 'SEASON_ID':season[0]}, ignore_index=True)

# #     except:    
# #         print(f"We've got an error when processing id={row['PLAYER_ID']}.")


In [9]:
# player_info can now be used to access different information that was returned by the request. CommonPlayerInfo contains the following data sets that are stored as a DataSet.

# available_seasons
# common_player_info
# player_headline_stats